In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def load_and_clean_data(file_path):
    """
    加载和清洗数据
    """
    data = pd.read_excel(file_path)
    data_cleaned = data.dropna(subset=['公司代码', '门店代码', '商品编码', '总净值', '数量'])
    data_cleaned = data_cleaned[data_cleaned['数量'] > 0]
    data_cleaned['单价'] = data_cleaned['总净值'] / data_cleaned['数量']
    return data_cleaned

# 建立回归模型
def build_regression_model(data):
    X = data[['单价']]  # 特征（价格）
    y = data['数量']    # 目标（销售量）

    # 划分训练集和测试集
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

    # 建立线性回归模型
    model = LinearRegression()
    model.fit(X_train, y_train)

    # 在测试集上评估模型
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    print(f"模型的均方误差（MSE）: {mse}")

    return model

# 预测不同价格点的销售量
def predict_sales(model, price_points):
    prices = np.array(price_points).reshape(-1, 1)
    predicted_sales = model.predict(prices)
    return dict(zip(price_points, predicted_sales))

# 主函数
def main():
    file_path = 'path_to_your_data.xlsx'
    sales_data = load_and_prepare_data(file_path)

    # 建立和评估模型
    model = build_regression_model(sales_data)

    # 预测不同价格点的销售量
    price_points = [20, 25, 30]  # 示例价格点
    predictions = predict_sales(model, price_points)

    for price, predicted_sale in predictions.items():
        print(f"价格 ${price} 的预测销售量: {predicted_sale}")

def analyze_sales_data(data):
    """
    分析销售数据
    """
    # 时间趋势分析
    data['月份'] = data['开票日期'].dt.to_period('M').dt.to_timestamp()
    monthly_sales_summary = data.groupby('月份').agg({'数量': 'sum', '总净值': 'sum'}).reset_index()

    plt.figure(figsize=(14, 6))
    plt.subplot(1, 2, 1)
    sns.lineplot(x='月份', y='数量', data=monthly_sales_summary, marker='o')
    plt.title('Total Sales Quantity Over Time')
    plt.xticks(rotation=45)
    plt.subplot(1, 2, 2)
    sns.lineplot(x='月份', y='总净值', data=monthly_sales_summary, marker='o')
    plt.title('Total Sales Value Over Time')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # 价格点分析
    price_sales_summary = data.groupby('单价').agg({'数量': 'sum', '总净值': 'sum'}).reset_index()
    return price_sales_summary

def suggest_pricing(price_sales_summary):
    """
    提出定价建议
    """
    # 根据销售量和销售额确定最佳价格点
    best_price = price_sales_summary.sort_values(by='总净值', ascending=False).iloc[0]['单价']
    print(f"建议的最佳定价点为: ${best_price}")

# 主函数
def main():
    file_path = 'path.xlsx'
    sales_data = load_and_clean_data(file_path)
    price_sales_summary = analyze_sales_data(sales_data)
    suggest_pricing(price_sales_summary)

if __name__ == "__main__":
    main()
